# Рекомендация тарифов

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Разделение данных на выборки

In [5]:
#Разделение на признаки и целевой признак
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [6]:
#Деление данных на обучающие и тестовые выборки
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345, stratify=target
)

print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)

(2571, 4)
(2571,)
(643, 4)
(643,)


In [7]:
#Отделение от обучающей выборки валидационной выборки
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train, target_train, test_size=0.25, random_state=12345, stratify=target_train
)
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)


## Исследование модели

<b>Решающее дерево</b>

In [8]:
best_result = 0
depth = 0
depths = []
results = []

for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    depths.append(depth)
    results.append(result)
    if result > best_result:
        best_depth = depth
        best_result = result

print('Accuracy:', best_result)
print('Лучшая глубина дерева:', best_depth)

Accuracy: 0.8164852255054432
Лучшая глубина дерева: 5


<b>Случайный лес</b>

In [9]:
best_model = None
best_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_est = est

print('Accuracy:', best_result)
print('Лучшее количество деревьев в лесу:', best_est)

Accuracy: 0.8133748055987559
Лучшее количество деревьев в лесу: 10


<b>Логистическая регрессия</b>

In [10]:
model = LogisticRegression()
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
accuracy = model.score(features_valid, target_valid)
print('Accuracy:', accuracy)

Accuracy: 0.7558320373250389


<b>Вывод</b>  

Лучший результат зафиксирован у решающего дерева - 81,64% при глубине дерева 5. У случайного леса - 81,33% при количестве деревьев в лесу 10. И наименьший результат у логистической регрессии - 75,58%.

## Проверка модели на тестовой выборке

In [11]:
model = DecisionTreeClassifier(random_state=12345, max_depth=5)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result = accuracy_score(target_test, predictions_test)
print('Accuracy тестовой выборки:', result)

Accuracy тестовой выборки: 0.7962674961119751


<b>Вывод</b>  

Результат на тестовой выборке 79,62% правильных ответов, на 2% меньше обучающей.

## Проверка модели на адекватность

In [12]:
dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(features_train, target_train)
dummy.predict(features_test)
dummy.score(features_test, target_test)

0.6936236391912908